In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
dev_ideal = qml.device("default.mixed", wires=2)  # no noise
dev_noisy = qml.transforms.insert(qml.DepolarizingChannel, 0.05, position="all")(
    dev_ideal
)

def U(angle):
    """A quantum function containing one parameterized gate.

    Args:
        angle (float): The phase angle for an IsingXY operator
    """
    qml.Hadamard(0)
    qml.Hadamard(1)
    qml.CNOT(wires=[0, 1])
    qml.PauliZ(1)
    qml.IsingXY(angle, [0, 1])
    qml.S(1)

@qml.qnode(dev_noisy)
def circuit(angle):
    """A quantum circuit made from the quantum function U.

    Args:
        angle (float): The phase angle for an IsingXY operator

    Returns:
        (numpy.array): A quantum state.
    """
    U(angle)
    return qml.state()

@qml.tape.stop_recording()
def circuit_ops(angle):
    """A function that outputs the operations within the quantum function U.

    Args:
        angle (float): The phase angle for an IsingXY operator

    Returns:
        (list(qml.operation.Operation)): A list of operations that make up the unitary U.
    """
    with qml.tape.QuantumTape() as tape:
        U(angle)
    return tape.operations

In [5]:
@qml.qnode(dev_noisy)
def global_fold_circuit(angle, n, s):
    """Performs the global circuit folding procedure.

    Args:
        angle (float): The phase angle for an IsingXY operator
        n: The number of times U^\dagger U is applied
        s: The integer defining L_s ... L_d.

    Returns:
        (numpy.array): A quantum state.
    """
    assert s <= len(
        circuit_ops(angle)
    ), "The value of s is upper-bounded by the number of gates in the circuit."

    U(angle)  # Original circuit application

    # Put your code here #

    # (U^\dagger U)^n
    for i in range(n):
#         qml.S(1).adjoint()
#         qml.IsingXY(angle, [0, 1]).adjoint()
#         qml.PauliZ(1).
#         qml.Hadamard(0)
#         qml.Hadamard(1)
#         qml.CNOT(wires=[0, 1])
#         qml.PauliZ(1)
        qml.adjoint(U)(angle)
        U(angle)

    # L_d^\dagger ... L_s^\dagger
    op = len(circuit_ops(angle))
    for i in range(op-1, s-2, -1):
        if i==5:
            qml.adjoint(qml.S(1))
        elif i==4:
            qml.adjoint(qml.IsingXY(angle, [0, 1]))
        elif i==3:
            qml.adjoint(qml.PauliZ(1))
        elif i==2:
            qml.adjoint(qml.CNOT(wires=[0, 1]))
        elif i==1:
            qml.adjoint(qml.Hadamard(1))
        elif i ==0:
            qml.adjoint(qml.Hadamard(0))
            
    # L_s ... L_d
    for i in range(s-1, op):
        if i==0:
            qml.Hadamard(0)
        elif i==1:
            qml.Hadamard(1)
        elif i==2:
            qml.CNOT(wires=[0, 1])
        elif i==3:
            qml.PauliZ(1)
        elif i==4:
            qml.IsingXY(angle, [0, 1])
        elif i==5:
            qml.S(1)
    
    return qml.state()

#fig, ax = qml.draw_mpl(global_fold_circuit)(0.5, 1, 4)
#fig.show()

In [6]:
def fidelity(angle, n, s):
    fid = qml.math.fidelity(global_fold_circuit(angle, n, s), circuit(angle))
    return np.round_(fid, decimals=5)


# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    angle, n, s = json.loads(test_case_input)
    fid = fidelity(angle, n, s)
    return str(fid)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your folded circuit isn't quite right!"


test_cases = [['[0.4, 2, 3]', '0.79209']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.4, 2, 3]'...
Correct!


/home/pritom/anaconda3/envs/lambeq/lib/python3.9/site-packages/pennylane/tape/stop_recording.py:85: UserWarning: qml.tape.stop_recording has moved to qml.QueuingManager.stop_recording.
  warn(


In [5]:
dev_ideal = qml.device("default.mixed", wires=2)  # no noise
@qml.qnode(dev_ideal)
def U(angle):
    """A quantum function containing one parameterized gate.

    Args:
        angle (float): The phase angle for an IsingXY operator
    """
    #qml.Hadamard(0)
    #qml.Hadamard(1)
    #qml.CNOT(wires=[0, 1])
    #qml.PauliZ(1)
    qml.IsingXY(angle, [0, 1])
    #qml.S(1)
    return qml.state()

In [14]:
op = qml.IsingXY(math.pi/4, wires=[0, 1]).adjoint()
qml.matrix(op)

array([[1.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.92387953+0.j        ,
        0.        -0.38268343j, 0.        +0.j        ],
       [0.        +0.j        , 0.        -0.38268343j,
        0.92387953+0.j        , 0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        0.        +0.j        , 1.        +0.j        ]])

In [24]:
qml.matrix(qml.S(1))

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+1.j]])

In [32]:
qml.matrix(qml.adjoint(qml.S(1)))

array([[1.-0.j, 0.-0.j],
       [0.-0.j, 0.-1.j]])

In [5]:
c = circuit_ops(0.5)
c

/home/pritom/anaconda3/envs/lambeq/lib/python3.9/site-packages/pennylane/tape/stop_recording.py:85: UserWarning: qml.tape.stop_recording has moved to qml.QueuingManager.stop_recording.
  warn(


[Hadamard(wires=[0]),
 Hadamard(wires=[1]),
 CNOT(wires=[0, 1]),
 PauliZ(wires=[1]),
 IsingXY(0.5, wires=[0, 1]),
 S(wires=[1])]

In [47]:
qml.ControlledQubitUnitary(qml.matrix(c[-2]), control_wires=[0], wires=[1]).adjoint()

ValueError: Input unitary must be of shape (2, 2) or (batch_size, 2, 2).

In [58]:
qml.matrix(qml.adjoint(qml.S(1)))

array([[1.-0.j, 0.-0.j],
       [0.-0.j, 0.-1.j]])

In [60]:
qml.matrix(qml.CNOT(wires=[0, 1]).adjoint())

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]])